# Basic analysis methods within the `spatialdata` ecosystem 

In this notebook, we will explore how to use `spatialdata-plot` to generate static plots of various different technologies. When we load the `spatialdata-plot` library, this adds the `.pl` accessor to every `SpatialData` object, which gives us access to the plotting functions. Their logic is loosly inspired by the ggplot-library in R, in which one can chain multiple function calls, gradually building the final figure.

In [14]:
import spatialdata as sd
import spatialdata_plot
import matplotlib.pyplot as plt  # for multi-panel plots later
import squidpy as sq

sdata_merfish = sd.read_zarr("../data/merfish.zarr")

In particular, the library exposes the following funcitons:
- [`Spatialdata.pl.render_images()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_images)
- [`Spatialdata.pl.render_labels()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_labels)
- [`Spatialdata.pl.render_points()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_points)
- [`Spatialdata.pl.render_shapes()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_shapes)
- [`Spatialdata.pl.show()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.show)

We can chain the 4 `render_xxx` functions to gradually build up a figure, with a final call to `show` to then actually render out the function. In the following sections we will explore these functions further.


## Overview
- [Simple function calls](#simple-viz)
- [Stacking function calls](#stacked-viz)
- [Transformations](#transformations)
- [Best practises](#best-practises)

## Simple function calls <a id="simple-viz"></a>
Let's first focus on the [MERFISH](https://vizgen.com/technology/) data from the previous notebook. As we can see below, it contains slots for `Images`, `Points`, `Shapes`, and `Tables`.

In [15]:
sdata_merfish

SpatialData object, with associated Zarr store: /Users/tim.treis/Documents/GitHub/spatialdata-workshops/data/merfish.zarr
├── Images
│     └── 'rasterized': DataArray[cyx] (1, 522, 575)
├── Points
│     └── 'single_molecule': DataFrame with shape: (<Delayed>, 3) (2D points)
├── Shapes
│     ├── 'anatomical': GeoDataFrame shape: (6, 1) (2D shapes)
│     └── 'cells': GeoDataFrame shape: (2389, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (2389, 268)
with coordinate systems:
    ▸ 'global', with elements:
        rasterized (Images), single_molecule (Points), anatomical (Shapes), cells (Shapes)

In [25]:
sdata_merfish.points["single_molecule"].compute()

,x,y,cell_type
0,1278.683956,6020.642260,outside_VISp
1,1326.970330,6023.884788,outside_VISp
2,1292.026844,6059.337093,outside_VISp
3,1300.886241,6097.786264,outside_VISp
4,1232.410068,6102.884182,outside_VISp
...,...,...,...
3714637,3161.427603,5192.594981,outside_VISp
3714638,3099.698528,5221.596008,outside_VISp
3714639,3084.582240,5297.234605,outside_VISp
3714640,3054.192051,5342.142346,VISp_II/III


Let's first visualise the individual contained modalities separately. 

## Transformations <a id="transformations"></a>
While this particular dataset is already (mostly) aligned correctly, we will use it to demonstrate the transformation logic of the `spatialdata` ecosystem. For the sake of this exercise, we will assume that the `Shapes` layer containing the anatomical regions needs to be adjusted to better match the "VISp_II/III" point annotation. We will use the `spatialdata` transformation logic to do so.

Using the [`spatialdata.transformations.get_transformations()`](https://spatialdata.scverse.org/en/latest/generated/spatialdata.transformations.get_transformation.html) function, we can extract the original transformation to the coordinate system it maps to:

In [17]:
sq.gr.spatial_neighbors(
    sdata_merfish,
    elements_to_coordinate_systems={"cells": "global"},
    table_key="table"
)
sdata_merfish.tables["table"]

AnnData object with n_obs × n_vars = 2389 × 268
    obs: 'cell_id', 'region'
    uns: 'spatialdata_attrs', 'spatial_neighbors'
    obsm: 'spatial'
    obsp: 'spatial_connectivities', 'spatial_distances'

In [20]:
sq.gr.nhood_enrichment(sdata_merfish, cluster_key="region")

ValueError: Expected at least `2` clusters, found `1`.